# (1) Backtest Statistics

## (1.1) Performance Metrics

In [28]:
def performance_backtest_metrics(positions) -> dict:
    
    """
    Calculate key backtest metrics:
    - Total PnL (still only have the gross, need to add qty and fees)
    - PnL from long positions  (until now only have long positions)
    - Annualized rate of return (until now only have a day of data)
    - Hit ratio
    - Average return from hits (don't have the qty)
    - Average return from misses (don't have the qty)
    """
    
    # Convert 'entry' and 'exit' column (dicts) into a DataFrame
    entry_df = pd.json_normalize(positions["entry"]) 
    exit_df = pd.json_normalize(positions["exit"])
    
    """
    # Add fees column if missing
    if 'fees' not in trades.columns:
        trades['fees'] = 0.002
    """

    # Calculate PnL per trade in dollars (still doesn't have the quantity) and doesn't have the fees
    gross = (exit_df["price"] - entry_df["price"])*positions["qty"]
    
    positions["pnl"] = gross 
    
    # --- Total PnL ---
    total_pnl = positions['pnl'].sum()
    
    # --- Hit ratio (fraction of profitable trades) ---
    hits = positions['pnl'] > 0
    hit_ratio = hits.mean()
    
     # --- Average return from hits and misses (percent relative to trade capital) ---
    # Trade capital approximated as entry_price * quantity
    positions['pct_return'] = positions['pnl'] / (entry_df["price"] * positions['qty'])

    avg_return_hits = positions.loc[hits, 'pct_return'].mean()
    avg_return_misses = positions.loc[~hits, 'pct_return'].mean()

    return {
        'Total_PnL_dollars': total_pnl,
        #'PnL_from_long_positions': long_pnl,
        #'Annualized_rate_of_return': annualized_return,
        'Hit_ratio': hit_ratio,
        'Average_return_from_hits': avg_return_hits,
        'Average_return_from_misses': avg_return_misses
    }


## (1.2) Runs (Returns Concentration)

In [29]:
import numpy as np

def return_concentration(returns):
    """
    Computes positive and negative return concentration (h+ and h-).
    
    Parameters:
        returns (array-like): Array of returns, can be positive or negative.
        
    Returns:
        h_plus, h_minus: Concentration measures for positive and negative returns.
    """
    returns = np.array(returns)
    
    # Separate positive and negative returns
    r_plus = returns[returns >= 0]
    r_minus = returns[returns < 0]
    
    # Helper function to compute concentration
    def concentration(r):
        n = len(r)
        if n <= 1:  # Not enough data to compute concentration
            return 0.0
        w = r / r.sum()  # normalized weights
        h = (np.sum(w**2) - 1/n) / (1 - 1/n)
        return h
    
    h_plus = concentration(r_plus)
    h_minus = concentration(r_minus)
    
    return h_plus, h_minus

## (1.3) Efficiency metrics

### (1.2.3) Sharpe Ratio

In [60]:
import pandas as pd
import numpy as np
from scipy.stats import norm, skew, kurtosis

def sharpe_ratio(daily_returns: pd.Series, risk_free_rate: float = 0.0361) -> float:
    
    """
    Compute annualized Sharpe ratio.
    daily_returns : pd.Series of daily returns 
    risk_free_rate: annual risk-free rate as decimal (e.g. 0.0361 for 3.61%%)
    """
    
    # Convert annual risk-free to daily
    daily_rf = (1 + risk_free_rate) ** (1/252) - 1
    excess = daily_returns - daily_rf
    mean_excess = excess.mean()
    std_excess = excess.std(ddof=1)

    return (mean_excess / std_excess) * np.sqrt(252)

def probabilistic_sharpe_ratio(returns: pd.Series, 
                                target_sr: float = 0.0, 
                                annualization_factor: int = 252) -> float:
    """
    Calculate the Probabilistic Sharpe Ratio (PSR).

    Parameters
    ----------
    returns : pd.Series
        Daily (or periodic) returns as decimals.
    target_sr : float, optional
        Sharpe ratio benchmark to test against (default=0.0)
    annualization_factor : int, optional
        Number of periods per year (daily=252, monthly=12, etc.)

    Returns
    -------
    float
        Probability (0-1) that the true Sharpe ratio exceeds the target.
    """
    # Compute sample statistics
    n = len(returns)
    mean_r = returns.mean()
    std_r = returns.std(ddof=1)
    
    # Compute annualized Sharpe ratio
    sr_hat = (mean_r / std_r) * np.sqrt(annualization_factor)
    
    if returns.std() < 1e-8:
        S, K = 0, 3  # assume normal distribution
    else:
        S = skew(returns)
        K = kurtosis(returns, fisher=False)
    
    # Standard error of Sharpe ratio adjusted for non-normality
    numerator = sr_hat - target_sr
    denominator = np.sqrt((1 - S * sr_hat + ((K - 1) / 4) * sr_hat**2) / n)
    
    # Probabilistic Sharpe Ratio (PSR)
    psr = norm.cdf(numerator / denominator)
    
    return psr

def deflated_sharpe_ratio(returns: pd.Series, 
                          N: int = 1, 
                          risk_free_rate: float = 0.0, 
                          annualization_factor: int = 252) -> float:
    """
    Calculate the Deflated Sharpe Ratio (DSR) as in Bailey & López de Prado (2014).

    Parameters
    ----------
    returns : pd.Series
        Daily (or periodic) returns as decimals.
    N : int
        Number of trials / strategies tested (accounts for selection bias)
    risk_free_rate : float
        Annual risk-free rate (e.g., 0.0361)
    annualization_factor : int
        Number of periods per year (daily=252, monthly=12)

    Returns
    -------
    float
        Deflated Sharpe Ratio probability (0-1)
    """
    # Convert returns to excess returns
    period_rf = risk_free_rate / annualization_factor
    excess_returns = returns - period_rf

    n = len(excess_returns)
    
    # Sample statistics
    mean_r = excess_returns.mean()
    std_r = excess_returns.std(ddof=1)
    
    # Annualized Sharpe
    sr_hat = (mean_r / std_r) * np.sqrt(annualization_factor)
    
    # Skewness and kurtosis
    S = skew(excess_returns)
    K = kurtosis(excess_returns, fisher=False)  # Pearson definition (K=3 for normal)
    
    # Standard error of Sharpe ratio (adjusted for skew/kurtosis)
    se_sr = np.sqrt((1 - S * sr_hat + ((K - 1) / 4) * sr_hat**2) / n)
    
    # Extreme value correction for N trials
    lambda_N = norm.ppf(1 - 1/N) if N > 1 else 0
    
    # Deflated Sharpe Ratio probability
    dsr = norm.cdf((sr_hat - lambda_N * se_sr) / se_sr)
    
    return dsr

# (2) Main Code

In [64]:
import json
import pandas as pd
import numpy as np
import os
from bson import json_util
from datetime import datetime
import asyncio
import nest_asyncio
import ipaddress
import pymongo
from datetime import datetime, timedelta
from statistics import mean, stdev
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, log_loss
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score

folders_positions = os.listdir("Positions")
folders_positions = [f for f in folders_positions if f != '.ipynb_checkpoints']
#files_positions = [files_tickdata[1]]
print(folders_positions)

['TickBars', 'VolumeImbalanceBars', 'VolumeBars']


In [65]:
for folder in folders_positions:
    
    positions_df = []
    results = []

    files_positions = os.listdir(f"Positions/{folder}")
    files_positions = [f for f in files_positions if f != '.ipynb_checkpoints']
    
    print("############################################################")
    print("############################################################")
    print(f"## {folder}")
    print("############################################################")

    for file in files_positions:

        print("\n")
        print("##############################################")
        print(f"## {file}")
        print("##############################################")
        print("\n")
        
        # ----------------------------------------------- #
        #  (1)             load data                      #
        # ----------------------------------------------- #

        # get the Y variables (labels)
        ## 1. Remove extension
        stem = Path(file).stem   # e.g "Signal_SMA_EMA"
        ## 2. Split by underscore
        parts = stem.split("_")  # ['Signal', 'SMA', 'EMA']
        ## 3. First part is the prefix ("Signal"), rest are suffixes
        prefix, suffixes = parts[0], parts[1:]
        ## 4. Rebuild into the desired list
        Y_variables = [f"{prefix} {s}" for s in suffixes]
        #print(Y_variables)

        # Load the match trades tick data
        with open(f"Positions/{folder}/{file}") as f:
            positions = pd.read_json(f)

        # ----------------------------------------------- #
        #  (1) End                                        #
        # ----------------------------------------------- #

        try:
            entry = positions.iloc[0]["entry"] #.iloc[0]["timestamp"])
            iso_string = (entry["timestamp"])["$date"]
            time_start  = datetime.fromisoformat(iso_string.replace("Z", "+00:00"))
            
            exit = positions.iloc[-1]["entry"]
            iso_string = (exit["timestamp"])["$date"]
            time_end = datetime.fromisoformat(iso_string.replace("Z", "+00:00")) 

            # Difference as a timedelta
            delta = time_end - time_start
            
            # Number of days (integer)
            days_between = delta.days  
            print(days_between)
            
        except:
            pass
        
        try: 
            # ----------------------------------------------- # 
            #  (2)              prepare data                  #
            
            # Convert 'change' column (dicts) into a DataFrame
            change_df = positions["change"]
            
            # (1.1.1) Time Range from all the positions.
            time_range = positions["trade duration"]

            # (1.1.2) Averaging holding period 
            Avg_hold_period = mean(time_range)
            
            print("\n")
            print("number of positions")
            print(len(positions))
            
            print("Average holding period (seconds)")
            print(Avg_hold_period)            

            # (1.2) Performance metrics #
            print("Performance metrics")
            print(performance_backtest_metrics(positions))

            # (1.3) Runs (Returns Concentration) #
            print("Runs (Returns Concentration)")
            h_plus, h_minus = return_concentration(change_df)
            print(f"h+ = {h_plus:.4f}, h- = {h_minus:.4f}")

            # (1.4) Efficiency metrics #
            
            # (1.4.1) Sharpe ratio #
            sharpe = sharpe_ratio(change_df, risk_free_rate=0.7673)
            print("Sharpe")
            print(sharpe)

            # (1.4.2) The Probabilistic Sharpe Ratio
            psr = probabilistic_sharpe_ratio(change_df, target_sr=0.0)
            print("Probabilistic Sharpe Ratio:")
            print(psr)
 
            # (1.4.3) Deflated Sharpe Ratio
            N_trials = 7
            dsr_value = deflated_sharpe_ratio(change_df, N=N_trials, risk_free_rate=0.0361)
            print("Deflated Sharpe Ratio")
            print(dsr_value)
            
            print("\n")
            
        except:
            pass

############################################################
############################################################
## TickBars
############################################################


##############################################
## Signal_SMA_EMA_RSI.json
##############################################




##############################################
## Signal_SMA.json
##############################################


1


number of positions
243
Average holding period (seconds)
89.54576954331418
Performance metrics
{'Total_PnL_dollars': 2577.530000000028, 'Hit_ratio': 0.5102880658436214, 'Average_return_from_hits': 0.00034228761431031343, 'Average_return_from_misses': -0.00016415823624440828}
Runs (Returns Concentration)
h+ = 0.0205, h- = 0.0080
Sharpe
-72.49753078283678
Probabilistic Sharpe Ratio:
0.9999999999998838
Deflated Sharpe Ratio
3.101673676585018e-11




##############################################
## Signal_EMA.json
##########################################

/tmp/ipykernel_18023/248454064.py:52: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  S = skew(returns)
/tmp/ipykernel_18023/248454064.py:53: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  K = kurtosis(returns, fisher=False)
/tmp/ipykernel_18023/248454064.py:101: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  S = skew(excess_returns)
/tmp/ipykernel_18023/248454064.py:102: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  K = kurtosis(excess_returns, fisher=False)  # Pearson definition (K=3 for n